In [1]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

In [2]:
# parametros cosecha
periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
lotes = []
penalizaciones = {
    'C1': [0.03, 0.02, 0.01, 0.005],
    'C2': [0.025, 0.012, 0.004, 0.002],
    'C3': [0.03, 0.02, 0.01, 0.005],
    'C4': [0.025, 0.012, 0.01, 0.002],
    'C5': [0.03, 0.02, 0.01, 0.005],
    'C6': [0.025, 0.012, 0.004, 0.002]
}

In [3]:
# Leo la base de datos y la guardo en una lista
c.execute("SELECT * FROM lotes")
lotes_obtenidos = c.fetchall()
lotes.append(lotes_obtenidos)
lotes = lotes[0]

In [4]:
#Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

# pasar lotes a lista de listas
lotes = [list(elem) for elem in lotes]
# Funcion que estima el dia de cosecha con media 0 y desv est 2
def dia_estimado():
        optimal_day = np.random.normal(0, 2)
        return int(round(optimal_day))
for lote in lotes:
    variacion_optimo = dia_estimado()
    lote[5]+=variacion_optimo
    lote.append(variacion_optimo)


In [5]:
# Lotes actualizados
print(lotes)

[[1, 'L_1_C4', 1, 'C4', 217.5, 116, 0.43, 0.39, 0.257, -2], [2, 'L_2_C4', 2, 'C4', 517.5, 125, 0.35, 0.44, 0.269, 2], [3, 'L_3_C6', 3, 'C6', 450.0, 78, 0.46, 0.38, 0.362, -1], [4, 'L_4_C4', 4, 'C4', 168.75, 120, 0.32, 0.51, 0.276, -2], [5, 'L_5_C6', 5, 'C6', 243.75, 109, 0.49, 0.6, 0.348, 5], [6, 'L_6_C3', 6, 'C3', 150.0, 96, 0.31, 0.56, 0.579, 0], [7, 'L_7_C5', 7, 'C5', 558.75, 110, 0.43, 0.39, 0.795, 1], [8, 'L_8_C6', 8, 'C6', 442.5, 77, 0.38, 0.35, 0.36, -1], [9, 'L_9_C1', 9, 'C1', 300.0, 125, 0.36, 0.35, 0.528, 0], [10, 'L_10_C5', 10, 'C5', 108.75, 81, 0.5, 0.56, 0.806, 2], [11, 'L_11_C1', 11, 'C1', 555.0, 101, 0.31, 0.53, 0.557, 1], [12, 'L_12_C1', 12, 'C1', 412.5, 126, 0.41, 0.33, 0.516, 2], [13, 'L_13_C5', 13, 'C5', 547.5, 90, 0.5, 0.48, 0.811, 1], [14, 'L_14_C2', 14, 'C2', 498.75, 117, 0.44, 0.52, 0.416, 1], [15, 'L_15_C1', 15, 'C1', 101.25, 125, 0.31, 0.48, 0.554, -2], [16, 'L_16_C6', 16, 'C6', 502.5, 99, 0.47, 0.34, 0.361, 2], [17, 'L_17_C5', 17, 'C5', 536.25, 114, 0.45, 0.39

In [6]:
# Función para predecir el clima con markov
def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
    clima_prediccion = []  # Lista para almacenar la predicción
    tipo_3 = []
    # Inicializar el primer día como seco
    clima_prediccion.append(0)
    tipo_3.append(0)
    # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
    for _ in range(7):
        if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
            clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
            tipo_3.append(0)
        else:  # Si el día anterior fue lluvioso
            clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
            if clima_prediccion[-1] == 1:
                tipo_3.append(1 if random.random() <= 0.05 else 0)
    if sum(tipo_3) >= 1:
        perdida = True
    else:
        perdida = False
    return clima_prediccion, perdida

In [7]:
def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
    lista_penalizaciones = penalizaciones[tipo_uva]
    index = 7
    while index != 0:
        if index == 7:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[0])
        elif index == 6:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[1])
        elif index == 5:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[2])
        elif index == 4:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 3:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 2:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 1:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        index -= 1
    return calidad


In [8]:
# funcion que entrega el valor final de la calidad considerando la lluvia
def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
    t_minus = -7
    t_plus = 7
    A = np.array([
        [t_minus**2, t_minus, 1],
        [0, 0, 1],
        [t_plus**2, t_plus, 1]])
    b = np.array([q_t_minus, 1, q_t_plus])
    coefficients = np.linalg.solve(A, b)
    a, b, c = coefficients
    def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
        # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
        calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
        calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
        return calidad_final
    q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
    return q_t_optimal

In [9]:
# Parametros para encontrar en la simulacion
calidades_lotes = []
lotes_perdidos = 0
lotes_obtenidos = []

In [10]:
# la simulación
index = 0
for t in range(50, 150):
    for lote in lotes:
        lote_id = lote[0]
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_dia_optimo = lote[5]
        lote_prob_seco_lluvia = lote[6]
        lote_prob_lluvia_lluvia = lote[7]
        lote_precio_usd = lote[8]
        lote_variacion = lote[9]
        if t == lote_dia_optimo - 8:
            lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
            if perdida == True:
                print("lote_perdido")
                lotes_perdidos += 1
                pass
            elif not perdida:
                q_t = calidades[lote_tipo_uva]
                lote_q_t_minus = q_t[0]
                lote_q_t_plus = q_t[1]
                calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
                calidades_lotes.append(calidad_lote)
                lotes_obtenidos.append(lote)
                lotes_obtenidos[index].append(calidad_lote)
                index += 1
                

lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido
lote_perdido


In [11]:
# Encontrar la info que queramos de esta etapa
print(lotes_perdidos)
print(calidades_lotes)
print(max(calidades_lotes))
print(min(calidades_lotes))
print(lotes_obtenidos)
print(len(lotes_obtenidos))

19
[0.9132653061224489, 0.90714208, 0.9387659505102041, 0.96032425, 0.995, 0.99, 0.9690783673469388, 0.9525735045918367, 0.98505, 0.9730127336734694, 0.933354503763, 0.96032425, 0.9949876693877551, 0.978155512703616, 1.0, 0.952402554860204, 0.9850178530612245, 0.9705093465918366, 0.9633195, 0.9754698140169861, 0.9670564199999999, 0.995, 0.9721794181224489, 0.9934034132293877, 0.9575279064, 0.977, 0.96032425, 0.9800756592979591, 0.9919794122448979, 0.9917805365077551, 0.9554984999999999, 0.982043682710204, 0.9718890612244898, 0.9735641705011296, 0.9471615544538265, 0.9809081632653062, 0.9904640816326531, 0.956746250966204, 0.9932005536391836, 0.97522412625, 0.9751241135204082, 0.9858622448979592, 0.9853390330873469, 0.9725049697959184, 0.9904640816326531, 0.9478106370688775, 0.9769686775510203, 0.998, 0.9853061224489796, 0.9779022448979592, 0.9550277814950037, 0.9665456572040817, 0.9242101522058623, 0.97034800561875, 0.9510830686665614, 0.9691372653061224, 0.9793156497069453, 0.94111776

In [12]:
# Guardar los lotes obtenidos en la base de datos
# Ejecutar solo una vez
import sqlite3

# Datos de ejemplo
lote = [175, 'L_175_C1', 175, 'C1', 517.5, 71, 0.41, 0.57, 0.55, 0, 0.965]

# Conectar a la base de datos (o crearla si no existe)
conn = sqlite3.connect('etapa_1.db')

### AQUI SACAR # PARA QUE FUNCIONE ESTA CELDA Y SE PUEDA CORRER
cursor = conn.cursor()

# Crear una tabla
cursor.execute('''
CREATE TABLE IF NOT EXISTS lotes (
    lote_id INTEGER PRIMARY KEY,
    lote_cod TEXT,
    lote_numero INTEGER,
    lote_tipo_uva TEXT,
    lote_toneladas REAL,
    lote_dia_optimo INTEGER,
    lote_prob_seco_lluvia REAL,
    lote_prob_lluvia_lluvia REAL,
    lote_precio_usd REAL,
    lote_variacion INTEGER,
    lote_calidad REAL
)
''')

# Insertar datos en la tabla
cursor.execute('''
INSERT INTO lotes (
    lote_id, lote_cod, lote_numero, lote_tipo_uva, lote_toneladas, 
    lote_dia_optimo, lote_prob_seco_lluvia, lote_prob_lluvia_lluvia, 
    lote_precio_usd, lote_variacion, lote_calidad
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', lote)

# Guardar los cambios y cerrar la conexión
conn.commit()
conn.close()


In [13]:
### Archivo para saber cómo leer 'etapa_1.db'

'''import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('etapa_1.db')
cursor = conn.cursor()

# Seleccionar todos los registros de la tabla 'lotes'
cursor.execute('SELECT * FROM lotes')

# Iterar sobre los resultados y imprimirlos
for lote in cursor.fetchall():
    lote_id, lote_cod, lote_numero, lote_tipo_uva, lote_toneladas, lote_dia_optimo, lote_prob_seco_lluvia, lote_prob_lluvia_lluvia, lote_precio_usd, lote_variacion, lote_calidad = lote
    print(f"Lote ID: {lote_id}, Código: {lote_cod}, Número: {lote_numero}, Tipo de Uva: {lote_tipo_uva}, Toneladas: {lote_toneladas}, Día Óptimo: {lote_dia_optimo}, Prob. Seco Lluvia: {lote_prob_seco_lluvia}, Prob. Lluvia Lluvia: {lote_prob_lluvia_lluvia}, Precio USD: {lote_precio_usd}, Variación: {lote_variacion}, Calidad: {lote_calidad}")

# Cerrar la conexión
conn.close()'''


'import sqlite3\n\n# Conectar a la base de datos\nconn = sqlite3.connect(\'etapa_1.db\')\ncursor = conn.cursor()\n\n# Seleccionar todos los registros de la tabla \'lotes\'\ncursor.execute(\'SELECT * FROM lotes\')\n\n# Iterar sobre los resultados y imprimirlos\nfor lote in cursor.fetchall():\n    lote_id, lote_cod, lote_numero, lote_tipo_uva, lote_toneladas, lote_dia_optimo, lote_prob_seco_lluvia, lote_prob_lluvia_lluvia, lote_precio_usd, lote_variacion, lote_calidad = lote\n    print(f"Lote ID: {lote_id}, Código: {lote_cod}, Número: {lote_numero}, Tipo de Uva: {lote_tipo_uva}, Toneladas: {lote_toneladas}, Día Óptimo: {lote_dia_optimo}, Prob. Seco Lluvia: {lote_prob_seco_lluvia}, Prob. Lluvia Lluvia: {lote_prob_lluvia_lluvia}, Precio USD: {lote_precio_usd}, Variación: {lote_variacion}, Calidad: {lote_calidad}")\n\n# Cerrar la conexión\nconn.close()'